# Informações gerais

## Tema
* Aplicação de *encoder* para RAG (*Retrieval Augmented Generation*)

## Integrantes
* Cesar Hideki Imai - 10402758
* João Victor Dallapé Madeira - 10400725
* David Varão Lima Bentes Pessoa - 1040264

# Setup

## Integração dos pacotes

O pacote `vectordb2` é usado para armazenar e recuperar textos usando técnicas de *chunking* (segmentação de texto), *embedding* (conversão de texto para vetores numéricos) e busca vetorial.

In [37]:
%pip install vectordb2

O pacote requests pode ser usado para recuperar o arquivo por meio de requisição em HTTP (esse pacote é opcional).

In [38]:
#%pip install requests

O pacote `networkx` é usado para a criação, manipulação e representação de grafos.

In [39]:
%pip install networkx

Importação das bibliotecas

In [40]:
from vectordb import Memory
#import requests
import networkx

## Structure

In [41]:
class Stack:
     def __init__(self):
         self.items = []

     def isEmpty(self):
         return self.items == []

     def push(self, item):
         self.items.append(item)

     def pop(self):
         return self.items.pop()

     def peek(self):
         return self.items[len(self.items)-1]

     def size(self):
         return len(self.items)

class Queue:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def enqueue(self, item):
        self.items.insert(0,item)

    def dequeue(self):
        return self.items.pop()

    def size(self):
        return len(self.items)

## Classe Grafo

In [2]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 13 13:59:10 2023

@author: icalc
"""

import sys

#CAMINHO = "palavras.txt"

# Grafo como uma matriz de adjacência
class Grafo:
    # construtor da classe grafo
    '''
    def __init__(self, path= CAMINHO):
        self.leGrafo(path)
    '''

	# Insere uma aresta no Grafo tal que
	# v é adjacente a w
    def insereA(self, v, w):
        if self.adj[v][w] == 0:
            self.adj[v][w] = 1
            self.m+=1 # atualiza qtd arestas

# remove uma aresta v->w do Grafo
    def removeA(self, v, w):
        if(v == w):
            return
        # testa se temos a aresta
        if self.adj[v][w] == 1:
            self.adj[v][w] = 0
            self.m -= 1  # atualiza qtd arestas

	# Apresenta o Grafo contendo
	# número de vértices, arestas
	# e a matriz de adjacência obtida
    def show(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                if self.adj[i][w] == 1:
                    print(f"Adj[{i:2d},{w:2d}] = 1 ", end="")
                else:
                    print(f"Adj[{i:2d},{w:2d}] = 0 ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )


	# Apresenta o Grafo contendo
	# número de vértices, arestas
	# e a matriz de adjacência obtida
    # Apresentando apenas os valores 0 ou 1
    def showMin(self):
        print(f"\n n: {self.n:2d} ", end="")
        print(f"m: {self.m:2d}\n")
        for i in range(self.n):
            for w in range(self.n):
                if self.adj[i][w] == 1:
                    print(" 1 ", end="")
                else:
                    print(" 0 ", end="")
            print("\n")
        print("\nfim da impressao do grafo." )


    # Função para calcular o grau de entrada no vértice v Ex 1)
    #OBS: Estou assumindo que, para os métodos abaixo, o valor v pertence ao intervalo [0, n - 1] ao invés de [1, n]
    def inDegree(self, v):
        degree = 0
        for i in range(self.n):
            degree += self.adj[i][v]
        return degree

    #Função para calcular o grau de saída no vértice v Ex 2)
    def outDegree(self, v):
        degree = 0
        for i in range(self.n):
            degree += self.adj[v][i]
        return degree

    #Função para verificar se um vértice é fonte Ex 3)
    def ehFonte(self, v):
        if(self.inDegree(v) == 0 and self.outDegree(v) > 0):
            return 1
        else:
            return 0

    #Função para verificar se um vértice é sorvedouro Ex 4)
    def ehSoverdouro(self, v):
        if(self.inDegree(v) > 0 and self.outDegree(v) == 0):
            return 1
        else:
            return 0

    #Função para verificar se um grafo é simétrico Ex 5)
    def ehSimetrico(self):
        for i in range(self.n):
            for j in range(i, self.n):
                if(self.adj[i][j] != self.adj[j][i]):
                    return 0
        return 1

    #Método para ler um grafo de um arquivo .txt Ex 6)
    '''
    def leGrafo(self, path= CAMINHO):
        with open(path, 'r', encoding='utf-8') as file:
            self.tipo = int(file.readline())
            self.n = int(file.readline())

            self.lista_nomes = [] #Insere nomes dos vértices
            for _ in range(self.n):
                self.lista_nomes.append(file.readline().split('"')[1])

            self.m = int(file.readline())
            # matriz de adjacência
            self.adj = [[0 for i in range(self.n)] for j in range(self.n)]
            for _ in range(self.m):
                string = file.readline()
                i, j, k = string.split(" ")
                i, j, k = int(i), int(j), float(k)
                self.adj[i][j] = k
    '''
    def leGrafo(self, path):
        with open(path, 'r',encoding='utf-8') as arquivo:
            n_palavras = int(arquivo.readline())

            palavras = []
            for i in range(n_palavras):
                palavras.append(arquivo.readline().strip())
        print(palavras)
        print(n_palavras)

    #Método para remover um vértice de um grafo Ex 9)
    def removeV(self, v):
        for i in range(self.n):
            if self.adj[i][v] == 1 or self.adj[v][i] == 1: #Verifica se havia uma aresta para subtrair
                self.m -= 1
            if i != v and i != len(self.adj) - 1: #Substitui a linha e coluna da matriz a serem apagadas pelas últimas linha e coluna respectivamente
                self.adj[i][v] = self.adj[i][-1]
                self.adj[v][i] = self.adj[-1][i]

        self.adj[v][v] = self.adj[-1][-1]
        for i in range(self.n):
            self.adj[i].pop() #Remove a última coluna da matriz

        self.adj.pop() #Remove a última linha
        self.n -= 1 #Decrementa um vértice

    def noAdjacente(self, n, nosMarcados):
        for i, item in zip(range(len(self.adj[n])), self.adj[n]):
            if item == 1 and i not in nosMarcados:
                return i
        return -1

    def EhAdjacente(self, v, x): #verifica se o vértice v é adjacente a x
        if self.adj[v][x] == 1:
            return True
        else:
            return False

    def percursoProfindidade(self, vInicio):
        percurso = [vInicio]
        nosMarcados = []
        pilha = Stack()

        nosMarcados.append(vInicio)
        pilha.push(vInicio)
        while(not pilha.isEmpty()):

            n = pilha.pop()
            while (self.noAdjacente(n, nosMarcados)) != -1:
                m = self.noAdjacente(n, nosMarcados)
                percurso.append(m)
                pilha.push(n)
                nosMarcados.append(m)
                n = m

        return percurso

    def percursoLargura(self, vInicio):
        percurso = [vInicio]
        nosMarcados = []
        fila = Queue()

        nosMarcados.append(vInicio)
        fila.enqueue(vInicio)
        while(not fila.isEmpty()):

            n = fila.dequeue()
            while(self.noAdjacente(n, nosMarcados) != -1):
                m = self.noAdjacente(n, nosMarcados)
                percurso.append(m)
                fila.enqueue(m)
                nosMarcados.append(m)

        return percurso

    def ordenacaotopologica(self):
        ge = [0] * self.n
        fila = Queue()
        ordenacao = []
        for i in range(self.n):
            ge[i] = self.inDegree(i)

        for i in range(self.n):
            if ge[i] == 0:
                fila.enqueue(i)
                ge[i] = -1

        while not fila.isEmpty():
            n = fila.dequeue()
            ordenacao.append(n) #visita nó
            for i in range(self.n):
                if self.EhAdjacente(n, i):
                    ge[i] -= 1

            for i in range(self.n):
                if ge[i] == 0:
                    fila.enqueue(i)
                    ge[i] = -1

        return ordenacao

    def substitui(self, vetor):
        for i in range(self.n):
            vetor[i] = chr(vetor[i] + 97) # Troca os índices da tabela pelas letras dos vértices
        return vetor

    def dijkstra(self, origem):
        # Inicializar distâncias com "infinito"
        origem -= 1
        distancias = [sys.maxsize] * self.n
        distancias[origem] = 0  # A distância da origem para ela mesma é 0

        # Conjunto de vértices visitados
        visitados = set()

        # Enquanto todos os vértices não forem visitados
        while len(visitados) < self.n:
            # Encontre o vértice com a menor distância que ainda não foi visitado
            min_distancia = sys.maxsize
            u = -1 # menor distância
            for i in range(self.n):
                if i not in visitados and distancias[i] < min_distancia:
                    min_distancia = distancias[i]
                    u = i

            # Marque o vértice `u` como visitado
            visitados.add(u)

            # Atualizar as distâncias dos vizinhos de `u`
            for v in range(self.n):
                if self.adj[u][v] > 0 and v not in visitados:  # Existe uma aresta entre `u` e `v`
                    nova_distancia = distancias[u] + self.adj[u][v]
                    if nova_distancia < distancias[v]:
                        distancias[v] = nova_distancia

        return distancias


# Métodos

## 1. Ler dados

### Aquisição dos dados

Os dados do documento são importados e guardados na variável `arquivo`.

In [43]:
'''
arquivo = requests.get('https://raw.githubusercontent.com/Cehiim/TeoriaDosGrafos/refs/heads/main/Projeto/palavras.txt').text

palavras = arquivo.split()
n_palavras = palavras.pop(0)
'''

"\narquivo = requests.get('https://raw.githubusercontent.com/Cehiim/TeoriaDosGrafos/refs/heads/main/Projeto/palavras.txt').text\n\npalavras = arquivo.split()\nn_palavras = palavras.pop(0)\n"

In [44]:
with open("./Projeto/palavras.txt", 'r') as arquivo:
  n_palavras = int(arquivo.readline())

  palavras = []
  for i in range(n_palavras):
    palavras.append(arquivo.readline().strip())

As palavras contidas no documento são segmentadas e armazenadas na lista `palavras`.

In [45]:
print(palavras)
print(n_palavras)

['Ecossistema', 'Sustentabilidade', 'Biodiversidade', 'Reciclagem', 'Conservação', 'Poluição', 'Desmatamento', 'Reflorestamento', 'Erosão', 'Compostagem', 'Biodegradável', 'Emissões', 'Pegada', 'Recursos', 'Preservação', 'Ecologia', 'Habitat', 'Fauna', 'Flora', 'Agroecologia', 'Bioma', 'Ciclo', 'Desenvolvimento', 'Economia', 'Efluentes', 'Gestão', 'Impacto', 'Mata', 'Amazônia', 'Cerrado', 'Pantanal', 'Biotecnologia', 'Agrofloresta', 'Agricultura', 'Aquicultura', 'Biocombustível', 'Solar', 'Eólica', 'Hidrelétrica', 'Resíduos', 'Saneamento', 'Tratamento', 'Uso', 'Zona', 'Proteção', 'Ambiental', 'Clima', 'Solo', 'Água', 'Floresta']
50


### Embedding das palavras

Aqui é utilizado a biblioteca VectorDB para criar uma memória virtual.

- `chunking_strategy`: Define a estratégia de fragmentação dos dados. No modo "sliding_window", os dados são divididos em *chunks* (pedaços de texto) de tamanho fixo.

- `window_size`: Define a quantidade de palavras que um *chunk* representa. Neste caso, cada *chunk* representa uma palavra.

- `overlap`: Define quantos elementos de sobreposição existirão entre os *chunks* adjacentes. Neste caso, não haverá sobreposição já que as palavras usadas não formam frases, logo são independentes uma das outras.

In [46]:
memory = Memory(chunking_strategy={"mode": "sliding_window", "window_size": 1, "overlap": 0})

for i in range(n_palavras):
  memory.save(palavras[i])

Initiliazing embeddings:  normal
OK.


Cada palavra é convertida para um vetor numérico e guardada na memória.

### Recuperação dos dados

Quanto menor é a distância, maior é a proximidade semântica.

In [47]:
for i in range(n_palavras):
  print(f"\n\nBusca: {palavras[i]}\n")
  buscas = memory.search(palavras[i], top_n=6)

  for j in range(1,6):
    palavra = buscas[j]['chunk']
    distancia = buscas[j]['distance']
    print(f"Palavra: {palavra}\nDistância: {distancia:.2f}\n")



Busca: Ecossistema

Palavra: Ecologia
Distância: 0.44

Palavra: Economia
Distância: 0.49

Palavra: Bioma
Distância: 0.58

Palavra: Biodiversidade
Distância: 0.58

Palavra: Sustentabilidade
Distância: 0.59



Busca: Sustentabilidade

Palavra: Biodiversidade
Distância: 0.45

Palavra: Economia
Distância: 0.56

Palavra: Ambiental
Distância: 0.56

Palavra: Ecossistema
Distância: 0.59

Palavra: Biotecnologia
Distância: 0.61



Busca: Biodiversidade

Palavra: Sustentabilidade
Distância: 0.45

Palavra: Agroecologia
Distância: 0.49

Palavra: Biotecnologia
Distância: 0.49

Palavra: Biodegradável
Distância: 0.53

Palavra: Bioma
Distância: 0.55



Busca: Reciclagem

Palavra: Compostagem
Distância: 0.56

Palavra: Reflorestamento
Distância: 0.64

Palavra: Recursos
Distância: 0.74

Palavra: Biotecnologia
Distância: 0.74

Palavra: Bioma
Distância: 0.76



Busca: Conservação

Palavra: Preservação
Distância: 0.24

Palavra: Proteção
Distância: 0.61

Palavra: Aquicultura
Distância: 0.77

Palavra: Poluiç

A palavra mais próxima armazenada na memória é ela mesma, portanto para encontrar as outras cinco palavras mais próximas foi recuperado as palavras de índice 1 até 6.

## 2. Gravar dados

## 3. Inserir vértice

## 4. Inserir aresta

## 5. Remover vértice

## 6. Remover aresta

## 7. Mostrar conteúdo

## 8. Mostrar grafo

## 9. Apresentar conexidade do grafo e o reduzido

# Menu

In [6]:
import os, time

grafo = Grafo() #Cria objeto grafo
fim = False
while(fim == False):
    print(
'''
Menu:
    1) Ler dados do arquivo em python
    2) Gravar dados no arquivo grafo.txt
    3) Inserir vértice
    4) Inserir aresta
    5) Remove vértice
    6) Remove aresta
    7) Mostrar conteúdo do arquivo
    8) Mostrar grafo
    9) Apresentar a conexidade do grafo e o reduzido
    10) Encerrar a aplicação
''')
    choice = int(input())
    if choice == 1: # Lê grafo
        grafo.leGrafo("./Projeto/palavras.txt")
        print("Grafo lido com sucesso!")

    elif choice == 2: # Grava dados no arquivo .txt
        print("oi")

    elif choice == 3: # Insere vértice
        print("oi")

    elif choice == 4: # Insere aresta
        print("oi")

    elif choice == 5: # Remove vértice
        print("oi")

    elif choice == 6: # Remove varesta
        print("oi")

    elif choice == 7: # Imprime arquivo
        print("oi")

    elif choice == 8: # Exibe grafo
        print("oi")

    elif choice == 9: # Apresenta a conexidade do grafo e grafo reduzido
        print("oi")

    elif choice == 10: # Encerra
        fim = True
        print("Encerrando programa...")

    else:
        print("Escolha inválida!")

    time.sleep(4) # Volta para o menu após 4 segundos

    if os.name == 'nt': # Limpa o terminal
        os.system('cls') # Caso o OS seja Windows
    else:
        os.system('clear') # Caso o OS seja Linux ou MacOS


Menu:
    1) Ler dados do arquivo em python
    2) Gravar dados no arquivo grafo.txt
    3) Inserir vértice
    4) Inserir aresta
    5) Remove vértice
    6) Remove aresta
    7) Mostrar conteúdo do arquivo
    8) Mostrar grafo
    9) Apresentar a conexidade do grafo e o reduzido
    10) Encerrar a aplicação

1
['Ecossistema', 'Sustentabilidade', 'Biodiversidade', 'Reciclagem', 'Conservação', 'Poluição', 'Desmatamento', 'Reflorestamento', 'Erosão', 'Compostagem', 'Biodegradável', 'Emissões', 'Pegada', 'Recursos', 'Preservação', 'Ecologia', 'Habitat', 'Fauna', 'Flora', 'Agroecologia', 'Bioma', 'Ciclo', 'Desenvolvimento', 'Economia', 'Efluentes', 'Gestão', 'Impacto', 'Mata', 'Amazônia', 'Cerrado', 'Pantanal', 'Biotecnologia', 'Agrofloresta', 'Agricultura', 'Aquicultura', 'Biocombustível', 'Solar', 'Eólica', 'Hidrelétrica', 'Resíduos', 'Saneamento', 'Tratamento', 'Uso', 'Zona', 'Proteção', 'Ambiental', 'Clima', 'Solo', 'Água', 'Floresta']
50
Grafo lido com sucesso!

Menu:
    1) Ler dad